In [32]:
import pandas as pd 
import numpy as np
from functools import lru_cache
import os
from tqdm import tqdm 

In [33]:
sasrec_UK_locale_prediction_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/predictions/SASRec_Next/kdd_cup_2023_UK/2023-04-16-21-09-54.parquet'
sasrec_DE_locale_prediction_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/predictions/SASRec_Next/kdd_cup_2023_DE/2023-04-16-21-12-54.parquet'
sasrec_JP_locale_prediction_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/predictions/SASRec_Next/kdd_cup_2023_JP/2023-04-16-21-15-12.parquet'

In [34]:
@lru_cache(maxsize=1)
def read_all_test_sessions():
    return pd.read_parquet('../predictions/SASRec_Next/kdd_cup_2023/2023-04-01-06-24-57.parquet', 'pyarrow')

@lru_cache(maxsize=1)
def read_sasrec_UK_locale_prediction():
    return pd.read_parquet(sasrec_UK_locale_prediction_path, engine='pyarrow')

@lru_cache(maxsize=1)
def read_sasrec_DE_locale_prediction():
    return pd.read_parquet(sasrec_DE_locale_prediction_path, engine='pyarrow')

@lru_cache(maxsize=1)
def read_sasrec_JP_locale_prediction():
    return pd.read_parquet(sasrec_JP_locale_prediction_path, engine='pyarrow')

In [35]:
sasrec_UK_locale_prediction = read_sasrec_UK_locale_prediction()
len(sasrec_UK_locale_prediction), len(sasrec_UK_locale_prediction.iloc[0]['next_item_prediction'])

(316971, 150)

In [36]:
sasrec_DE_locale_prediction = read_sasrec_DE_locale_prediction()
len(sasrec_DE_locale_prediction), len(sasrec_DE_locale_prediction.iloc[0]['next_item_prediction'])

(316971, 150)

In [37]:
sasrec_JP_locale_prediction = read_sasrec_JP_locale_prediction()
len(sasrec_JP_locale_prediction), len(sasrec_JP_locale_prediction.iloc[0]['next_item_prediction'])

(316971, 150)

In [38]:
len(sasrec_DE_locale_prediction[sasrec_DE_locale_prediction['locale'] == 'DE'])

104568

In [39]:
len(sasrec_JP_locale_prediction[sasrec_JP_locale_prediction['locale'] == 'JP'])

96467

In [40]:
len(sasrec_UK_locale_prediction[sasrec_UK_locale_prediction['locale'] == 'UK'])

115936

In [41]:
new_test_prediction = pd.concat([sasrec_DE_locale_prediction.iloc[ : 104568], sasrec_JP_locale_prediction.iloc[104568 : 104568 + 96467], sasrec_UK_locale_prediction.iloc[104568 + 96467: ]], axis=0)

In [42]:
def new_pred_validation(new_pred):
    assert len(new_pred) == len(sasrec_DE_locale_prediction)
    assert new_pred.iloc[104567]['locale'] == 'DE'
    assert new_pred.iloc[104568]['locale'] == 'JP'
    assert new_pred.iloc[104568 + 96467 - 1]['locale'] == 'JP'
    assert new_pred.iloc[104568 + 96467]['locale'] == 'UK'
    print("Nice!!!")

In [43]:
new_pred_validation(new_test_prediction)

Nice!!!


In [44]:
new_test_prediction['next_item_prediction'].apply(len).describe()

count    316971.0
mean        150.0
std           0.0
min         150.0
25%         150.0
50%         150.0
75%         150.0
max         150.0
Name: next_item_prediction, dtype: float64

In [19]:
new_test_prediction.to_parquet('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/predictions/SASRec_Next/kdd_cup_2023/three_locale_prediction_0416_2120.parquet', engine='pyarrow')